In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dill
from pytictoc import TicToc
from IPython.display import clear_output
from cec2013lsgo.cec2013 import Benchmark

In [2]:
def DSRegPSO(D,n,Ll,Lu,c1,c2,Mmax,lambd,fdmax,fdmin,Smax,Smin,dseta,fevalmax,run,function,f,info):
    LSui = lambd*(Lu-Ll)
    LSu = LSui
    LSl = -LSu
    w = Mmax/2
    X = np.random.uniform(Ll,Lu,(n,D))
    V = np.zeros(X.shape)
    C = np.array([f(X[i,:])for i in range(X.shape[0])])
    feval = n
    GPi = C
    Pi = X
    GBid = np.argmin(GPi)
    GB = GPi[GBid]
    PG = X[GBid]
    GB0 = GB
    Ss = Smin
    deltamax = np.max(np.abs(PG))*fdmax
    deltamin = np.min(np.abs(PG))*fdmin
    delta = deltamin
    it = 0
    GBlist = [[]]*int(fevalmax/n)
    GBlist[it] = GB 
    t = TicToc()
    t.tic()
    while feval+n <= fevalmax:
        R1 = np.random.random(X.shape)
        R2 = np.random.random(X.shape)
        V = ((delta/deltamax)+(Ss/Smax))*w*V+c1*R1*(PG-X)+c2*R2*(Pi-X)
        V = np.clip(V, LSl,LSu)
        deltai = np.sum(np.absolute(X-PG),axis = 1)
        U = np.repeat((deltai < delta).reshape(n,1),D,axis = 1)
        psi = np.random.uniform(Ll,Lu,(n,D))
        X = (1-U)*(X+V)+U*psi
        X[X>Lu] = np.maximum(Ll,Lu+(Lu-X[X>Lu]))
        X[X<Ll] = np.minimum(Lu,Ll+Ll-X[X<Ll])
        X = np.clip(X, Ll,Lu)
        C = np.array([f(X[i,:]) for i in range(X.shape[0])])
        feval += n
        for i in range(n):
            if C[i]<GPi[i]:
                GPi[i]=C[i]
                Pi[i,:] = X[i,:]
                if GPi[i]<GB:
                    GB = GPi[i]
                    PG = Pi[i,:]
               
        it+=1
        
        if GB == 0:
            time = t.tocvalue()
            clear_output(wait=True)
            print('Function',function,'Iteration',f"{it:.4E}",'Function Evaluations',f"{(feval):.4E}", 'Best Cost Now', f"{GB:.4E}",'Expected',info['best'] ,'Time', time)
            print('Run',run)
            print('n',n)
            print('dseta',dseta)
            print('fdmax',fdmax)
            print('fdmin',fdmin)
            print('LSu',LSu)
            print('LSl',LSl)
            print('delta',delta)
            print('deltamin',deltamin)
            print('deltamax',deltamax)
            print('Ss',Ss)
            return GBlist,GB,PG
        
        if np.absolute(GB-GB0)<=dseta*GB:
            if delta<deltamax:
                delta = delta+deltamax*Ss
            else:
                delta = deltamin
                if Ss>=Smax:
                    Ss = Smin
                else:
                    Ss = Ss+Smin        
        else:
            delta = deltamin
            Ss = Smin
            
        deltamax = np.max(np.abs(PG))*fdmax
        deltamin = np.min(np.abs(PG))*fdmin
        LSu = (((delta/deltamax)+(Ss/Smax))/2)*(LSui)
        LSl = -LSu
        GB0 = GB
        GBlist[it] = GB 
        if np.mod(feval,10000)==0:
            time = t.tocvalue()
            clear_output(wait=True)
            print('Function',function,'Iteration',f"{it:.4E}",'Function Evaluations',f"{(feval):.4E}", 'Best Cost Now', f"{GB:.4E}",'Expected',info['best'] ,'Time', time)
            print('Run',run)
            print('n',n)
            print('dseta',dseta)
            print('fdmax',fdmax)
            print('fdmin',fdmin)
            print('LSu',LSu)
            print('LSl',LSl)
            print('delta',delta)
            print('deltamin',deltamin)
            print('deltamax',deltamax)
            print('Ss',Ss)
            t.tic()
    return GBlist,GB,PG

In [3]:
if __name__ == '__main__':
    DATA = pd.DataFrame({'alg':['DSRegPSOP']*3,
              'milestone':[1.2e5,6.0e5,3.0e6],
                'f1':[float('inf')]*3,'f2':[float('inf')]*3,'f3':[float('inf')]*3,'f4':[float('inf')]*3,'f5':[float('inf')]*3,
                'f6':[float('inf')]*3,'f7':[float('inf')]*3,'f8':[float('inf')]*3,'f9':[float('inf')]*3,
                'f10':[float('inf')]*3,'f11':[float('inf')]*3,'f12':[float('inf')]*3,'f13':[float('inf')]*3,'f14':[float('inf')]*3,
                'f15':[float('inf')]*3})
    DATA2 = pd.DataFrame({'Function':['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12','f13','f14','f15'],
                          'Median':[[]]*15,'Mean':[[]]*15,'Best':[[]]*15,'Worst':[[]]*15,'SD':[[]]*15})
    function = 1 #######
    
    
     
    
    #########################
    
    Mmax = 0.0  
    Smax = 1.5
    Smin = 0.02
    c1 = 2
    c2 = 1.5
    dseta = 0.01
    fdmax = 1.0e-200
    fdmin = 1e-200
    lambd = 0.7
    n = 50
         
    #########################
    
    fevalmax = 3e6
    runs = 1
    np.random.seed(0)
    tt = TicToc()
    config = []
    load = False ###############################
    i= 0
    
    run_list = []
    BEST1_2_5 = []
    BEST6_0_5 = []
    BEST3_0_6 = []
    j = 0
    while j < runs:
        if load:
            dill.load_session('./tmp/function_'+str(function)+'_'+'saveworkspace.pkl')
            load = False
            i-=1
            j-=1
        itmax = int(3e6/n)##################################
        bench = Benchmark()
        info = bench.get_info(function)
        f = bench.get_function(function)
        D = info['dimension']
        Lu = info['upper']
        Ll = info['lower']
        tt.tic()
        GBlist,GB1,X1 = DSRegPSO(D,n,Ll,Lu,c1,c2,Mmax,lambd,fdmax,fdmin,
                            Smax,Smin,dseta,fevalmax, j+1,function,f,info)##########################
        BEST1_2_5.append(GBlist[int(1.2e5/n)-1])#########################
        BEST6_0_5.append(GBlist[int(6.0e5/n)-1])#########################
        BEST3_0_6.append(GB1)
        dill.dump_session('./tmp/function_'+str(function)+'_'+'saveworkspace.pkl')
        tt.toc()
        j+=1
        del bench, info, f
        run_list.append(GB1)
    mean_runs = np.mean(run_list)
    DATA2.loc[function-1,'Best'] = BEST3_0_6
    DATA2.loc[function-1,'Median'] = [np.median(BEST1_2_5),np.median(BEST6_0_5),np.median(BEST3_0_6)]
    DATA2.loc[function-1,'Mean'] = [np.mean(BEST1_2_5),np.mean(BEST6_0_5),np.mean(BEST3_0_6)]
    DATA2.loc[function-1,'Best'] = [np.min(BEST1_2_5),np.min(BEST6_0_5),np.min(BEST3_0_6)]
    DATA2.loc[function-1,'Worst'] = [np.max(BEST1_2_5),np.max(BEST6_0_5),np.max(BEST3_0_6)]
    DATA2.loc[function-1,'SD'] = [np.std(BEST1_2_5),np.std(BEST6_0_5),np.std(BEST3_0_6)]
    i+=1           

Function 1 Iteration 1.2999E+04 Function Evaluations 6.5000E+05 Best Cost Now 5.4158E+05 Expected 0.0 Time 2.299946190999435
Run 1
n 50
dseta 0.01
fdmax 1e-200
fdmin 1e-200
LSu 21.469891536297393
LSl -21.469891536297393
delta 4.6068720761350786e-203
deltamin 4.6068720761350786e-203
deltamax 9.999816496686442e-199
Ss 0.46000000000000013


In [ ]:
import matplotlib.pyplot as plt

In [ ]:

fig = plt.figure(1)
ax = fig.add_subplot(111)
ax.plot(GBlist)
ax.set_title(f"Global Best Cost {min(GBlist):.2E}")
ax.set_xlabel("Function Evaluations")
ax.set_ylabel("Cost Value")
plt.show()


In [ ]:
DATA2

In [ ]:
DATA.loc[0,'f'+str(function)] = DATA2['Best'][function-1][0]
DATA.loc[1,'f'+str(function)] = DATA2['Best'][function-1][1]
DATA.loc[2,'f'+str(function)] = DATA2['Best'][function-1][2]


In [ ]:
DATA

In [ ]:
dill.dump_session('./tmp/function_'+str(function)+'_'+'saveworkspace.pkl')

In [ ]:
DATA.to_csv('BEST_f'+str(function)+'_final.csv',index=False)
DATA2.to_csv('Results_f'+str(function)+'_final.csv',index=False)